In [3]:
import numpy as np
import keras
from keras.layers import Dense
from keras.models import Sequential
from matplotlib import pyplot as plt
import time

In [4]:
# load the model 
model_2 = keras.models.load_model("x1_smaller_than_x2.h5")

2021-12-01 13:50:30.769788: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-01 13:50:30.770306: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-01 13:50:30.774359: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [23]:
# prepare the required parameters and load the functions need to be used
%run single_layer_interval_propagation.ipynb
%run simple_interval_propagation.ipynb
%run model_2_functions.ipynb
# the output should be all less than 0.5 because all the x1 is larger than x2 in this interval. each valus in maximum vector should less than 0,5
# set the initial input region
min_vec = np.array([[50., 0.1]])
max_vec = np.array([[90., 49.]])
initial_region = [min_vec, max_vec]

In [18]:
# TODO list is first in first out, larger interval has priority.
# get the start time
start_time=time.time()
# set the time out seconds. if this time is reached, it will stop to verify the intervals in the TODO list.
total_seconds = 2

# initial a first-in-first-out list, it only has the first region we want to check. 
list_fifo = []
list_fifo.append(initial_region)
list_fifo_log=[]
list_fifo_log.append(initial_region)
# list to store all counter examples
counter_examples = []

while True:
    
    # update elapsed_time
    elapsed_time = time.time() - start_time
    
    #check if the list_fifo is empty, if it is empty, all intervals have passed the check
    if not list_fifo:
        print("the interval certification success! use "+str(elapsed_time)+" seconds! success!")
        break # break the main loop
        
    if elapsed_time > total_seconds:
        print("time out! Searching the counter examples in the TODO list! "+str(len(list_fifo))+" intervals in TODO list!")
        # before return time out, check if the regions in the list has counter examples?
        counter_examples = if_any_counter_example_model_2_large_first(model_2, list_fifo)
        if counter_examples:
            # if there is counter examples, print counter examples
            print("Found "+str(len(counter_examples))+" counter examples: ")
            for example in counter_examples:
                print(str(example))
            break
        else:
            # if can not found counter examples, print failed notification
            print("Fail to certify the nn model with the given region and specification! use "+str(elapsed_time)+" seconds! And cannot found counter example!")
            break
    
    
    input_region = list_fifo.pop(0)
    input_min = input_region[0]
    input_max = input_region[1]
    
    outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
    
    # check if the output(outputMin,outputMax) is satisfy the specification
    # for this example is all values in outputMax is less than 0.5
    if not check_specification_model_2(outputMax):
        # split into two intervals, and add these two intervals into list_fifo
        region1=[]
        region2=[]
        region1, region2 = split_2(input_min, input_max)
        list_fifo.append(region1)
        list_fifo.append(region2)
        list_fifo_log.append(region1)
        list_fifo_log.append(region2)
print(len(list_fifo_log))
print(list_fifo_log)
for region in list_fifo_log:
    input_min = input_region[0]
    input_max = input_region[1]
    outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
    print(outputMax)

time out! Searching the counter examples in the TODO list! 1220 intervals in TODO list!
Found 1 counter examples: 
[[50.62255859 40.78632812]]


In [22]:
# ouput the 4 round in main for loop, actually it is 3 round.
# first is to check min(70, 7), max(90, 20)，because the output maximum value is 0.99. 
# then split the region into two part region1[min(70, 7); max(80,20)]; region2[min(80,7);max(90,20)]
# these two regions' output maximum value are both smaller than 0.5
min_vec = np.array([[70., 7.]])
max_vec = np.array([[90., 20.]])
outputMin, outputMax = simple_interval_propagation_model(model_2, min_vec, max_vec)
print(outputMin)
print(outputMax)
min_vec = np.array([[70.], [7.]])
max_vec = np.array([[80.], [20.]])
outputMin, outputMax = simple_interval_propagation_model(model_2, min_vec, max_vec)
print(outputMin)
print(outputMax)
min_vec = np.array([[80.], [7.]])
max_vec = np.array([[90.], [20.]])
outputMin, outputMax = simple_interval_propagation_model(model_2, min_vec, max_vec)
print(outputMin)
print(outputMax)

[[1.51758515e-13]]
[[0.99043261]]
[[6.57964252e-12]]
[[0.26144672]]
[[2.92889482e-13]]
[[0.0694856]]


In [12]:
# TODO list is last in first out, smaller interval have prioruty 
# get the start time
start_time=time.time()
# set the time out seconds. if this time is reached, it will stop to verify the intervals in the TODO list.
total_seconds = 2

# initial a first-in-first-out list, it only has the first region we want to check. 
list_lifo = []
list_lifo.append(initial_region)
# list to store all counter examples
counter_examples = []

while True:
    
    # update elapsed_time
    elapsed_time = time.time() - start_time
    
    #check if the list_lifo is empty, if it is empty, all intervals have passed the check
    if not list_lifo:
        print("the interval certification success! use "+str(elapsed_time)+" seconds! success!")
        break # break the main loop
        
    if elapsed_time > total_seconds:
        print("time out! Searching the counter examples in the TODO list! "+str(len(list_lifo))+" intervals in TODO list!")
        # before return time out, check if the regions in the list has counter examples?
        counter_examples = if_any_counter_example_model_2_small_first(model_2, list_lifo)
        if counter_examples:
            # if there is counter examples, print counter examples
            print("Found "+str(len(counter_examples))+" counter examples: ")
            for example in counter_examples:
                print(str(example))
            break
        else:
            # if can not found counter examples, print failed notification
            print("Fail to certify the nn model with the given region and specification! use "+str(elapsed_time)+" seconds! And cannot found counter example!")
            break
    
    
    input_region = list_lifo.pop(len(list_lifo)-1)
    input_min = input_region[0]
    input_max = input_region[1]
    
    outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
    
    # check if the output(outputMin,outputMax) is satisfy the specification
    # for this example is all values in outputMax is less than 0.5
    if not check_specification_model_2(outputMax):
        # split into two intervals, and add these two intervals into list_lifo
        region1=[]
        region2=[]
        region1, region2 = split_2(input_min, input_max)
        list_lifo.append(region1)
        list_lifo.append(region2)

time out! Searching the counter examples in the TODO list! 83 intervals in TODO list!
Found 1 counter examples: 
[[70.96574605 49.        ]]


2021-11-08 18:14:48.387055: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-08 18:14:48.403805: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3193825000 Hz


In [16]:
print(model_2.predict(counter_examples[0]))

[[0.9442816]]


In [14]:
# prepare the required parameters and load the functions need to be used
%run single_layer_interval_propagation.ipynb
%run simple_interval_propagation.ipynb
%run model_2_functions.ipynb

In [8]:
# in 5 intervals,each interval get 200 random points. l infinity perturbation to get 1000 intervals. 
# how many intervals can pass the certification, when l infinity norm is 0.5, 1.0, 1.5, 2.0, 2.5
min_vec_1 = [50., 7.]
max_vec_1 = [70., 20.]
check_interval_1 = [min_vec_1, max_vec_1]
orig_points_1 = get_point_list_model_2(200, check_interval_1)

min_vec_2 = [80., 1.]
max_vec_2 = [97., 77.]
check_interval_2 = [min_vec_2, max_vec_2]
orig_points_2 = get_point_list_model_2(200, check_interval_2)

min_vec_3 = [10., 2.]
max_vec_3 = [47., 7.]
check_interval_3 = [min_vec_3, max_vec_3]
orig_points_3 = get_point_list_model_2(200, check_interval_3)

# y_true == 1
min_vec_4 = [1., 70.]
max_vec_4 = [67., 90.]
check_interval_4 = [min_vec_4, max_vec_4]
orig_points_4 = get_point_list_model_2(200, check_interval_4)

min_vec_5 = [1., 40.]
max_vec_5 = [30., 60.]
check_interval_5 = [min_vec_5, max_vec_5]
orig_points_5 = get_point_list_model_2(200, check_interval_5)

orig_point_list_600 = orig_points_1 + orig_points_2 + orig_points_3
orig_points_list_400 = orig_points_4 + orig_points_5

In [38]:
# 0.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 0.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.183225393295288
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [39]:
# 1.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.2229838371276855
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [40]:
# 1.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

21.511252641677856
{'sat_num': 2, 'counter_interval_list': [[array([[78.50683442, 73.73369533]]), array([[81.50683442, 76.73369533]])], [array([[8.51487492, 5.38379575]]), array([[11.51487492,  8.38379575]])]], 'counter_examples': [[array([[78.65038911, 76.2473672 ]])], [array([[8.5778632 , 8.02930356]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [43]:
for i in results_600["counter_examples"]:
    print(model_2.predict(i))

[[0.50030744]]
[[0.5410635]]


In [41]:
# 2.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

52.52793073654175
{'sat_num': 5, 'counter_interval_list': [[array([[79.22356275, 72.51300108]]), array([[83.22356275, 76.51300108]])], [array([[78.00683442, 73.23369533]]), array([[82.00683442, 77.23369533]])], [array([[8.58538599, 4.64718617]]), array([[12.58538599,  8.64718617]])], [array([[8.7964178 , 4.73985106]]), array([[12.7964178 ,  8.73985106]])], [array([[8.01487492, 4.88379575]]), array([[12.01487492,  8.88379575]])]], 'counter_examples': [[array([[79.23412184, 76.51092588]])], [array([[79.51464692, 77.15557033]])], [array([[9.26116724, 8.63546742]])], [array([[8.84524592, 8.69688231]])], [array([[8.67893742, 8.18848325]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [45]:
for i in results_600["counter_examples"]:
    print(model_2.predict(i))

[[0.50030744]]
[[0.5410635]]
[[0.50066483]]
[[0.54997545]]
[[0.5135811]]


In [46]:
# 2.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

93.20939946174622
{'sat_num': 9, 'counter_interval_list': [[array([[78.72356275, 72.01300108]]), array([[83.72356275, 77.01300108]])], [array([[77.86592169, 70.54573605]]), array([[82.86592169, 75.54573605]])], [array([[77.50683442, 72.73369533]]), array([[82.50683442, 77.73369533]])], [array([[8.17666626, 3.25072157]]), array([[13.17666626,  8.25072157]])], [array([[8.21657744, 2.89417766]]), array([[13.21657744,  7.89417766]])], [array([[7.6062021 , 2.58572457]]), array([[12.6062021 ,  7.58572457]])], [array([[8.08538599, 4.14718617]]), array([[13.08538599,  9.14718617]])], [array([[8.2964178 , 4.23985106]]), array([[13.2964178 ,  9.23985106]])], [array([[7.51487492, 4.38379575]]), array([[12.51487492,  9.38379575]])]], 'counter_examples': [[array([[78.86516432, 76.98858702]])], [array([[77.89643927, 75.40169308]])], [array([[77.79003755, 76.97197658]])], [array([[8.42324829, 7.99193251]])], [array([[8.23488799, 7.72205852]])], [array([[7.66723726, 7.24148629]])], [array([[8.14886255

In [47]:
for i in results_600["counter_examples"]:
    print(model_2.predict(i))

[[0.5849297]]
[[0.5209565]]
[[0.6460297]]
[[0.51922053]]
[[0.5096723]]
[[0.51735216]]
[[0.51339513]]
[[0.5435649]]
[[0.5492661]]


In [9]:
# 3.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.99
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_with_100_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

2021-12-01 13:52:26.735030: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-01 13:52:26.754515: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3194025000 Hz


144.88659858703613
{'sat_num': 14, 'counter_interval_list': [[array([[77.27896113, 68.64722976]]), array([[83.25896113, 74.62722976]])], [array([[78.23356275, 71.52300108]]), array([[84.21356275, 77.50300108]])], [array([[77.37592169, 70.05573605]]), array([[83.35592169, 76.03573605]])], [array([[77.01683442, 72.24369533]]), array([[82.99683442, 78.22369533]])], [array([[7.01423187, 0.52166286]]), array([[12.99423187,  6.50166286]])], [array([[8.02334095, 2.36233755]]), array([[14.00334095,  8.34233755]])], [array([[7.68666626, 2.76072157]]), array([[13.66666626,  8.74072157]])], [array([[7.72657744, 2.40417766]]), array([[13.70657744,  8.38417766]])], [array([[7.1162021 , 2.09572457]]), array([[13.0962021 ,  8.07572457]])], [array([[7.59538599, 3.65718617]]), array([[13.57538599,  9.63718617]])], [array([[9.67304835, 3.84638165]]), array([[15.65304835,  9.82638165]])], [array([[7.8064178 , 3.74985106]]), array([[13.7864178 ,  9.72985106]])], [array([[7.5847312 , 1.97740704]]), array([

In [ ]:
for i in results_600["counter_examples"]:
    print(model_2.predict(i))

In [33]:
# prepare the required parameters and load the functions need to be used
%run single_layer_interval_propagation.ipynb
%run simple_interval_propagation.ipynb
%run model_2_functions.ipynb

In [34]:
#***************************************************larger pri search smaller**********************************************************
# 0.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 0.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

1.2640185356140137
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [35]:
# 1.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

1.3308134078979492
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [36]:
# 1.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

21.64688992500305
{'sat_num': 2, 'counter_interval_list': [[array([[78.50683442, 73.73369533]]), array([[81.50683442, 76.73369533]])], [array([[8.51487492, 5.38379575]]), array([[11.51487492,  8.38379575]])]], 'counter_examples': [[array([[79.13671724, 76.6223672 ]])], [array([[8.8591132 , 8.33985043]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
[[0.52287304]]
[[0.51101494]]


In [37]:
# 2.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

52.39330816268921
{'sat_num': 5, 'counter_interval_list': [[array([[79.22356275, 72.51300108]]), array([[83.22356275, 76.51300108]])], [array([[78.00683442, 73.23369533]]), array([[82.00683442, 77.23369533]])], [array([[8.58538599, 4.64718617]]), array([[12.58538599,  8.64718617]])], [array([[8.7964178 , 4.73985106]]), array([[12.7964178 ,  8.73985106]])], [array([[8.01487492, 4.88379575]]), array([[12.01487492,  8.88379575]])]], 'counter_examples': [[array([[79.22911695, 76.5126959 ]])], [array([[79.01464692, 76.69463283]])], [array([[8.95062036, 8.64327992]])], [array([[9.04055842, 8.41758543]])], [array([[8.35471867, 8.82910825]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
[[0.5010346]]
[[0.5436866]]
[[0.53450024]]
[[0.50023973]]
[[0.58755785]]


In [38]:
# 2.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

93.60594153404236
{'sat_num': 9, 'counter_interval_list': [[array([[78.72356275, 72.01300108]]), array([[83.72356275, 77.01300108]])], [array([[77.86592169, 70.54573605]]), array([[82.86592169, 75.54573605]])], [array([[77.50683442, 72.73369533]]), array([[82.50683442, 77.73369533]])], [array([[8.17666626, 3.25072157]]), array([[13.17666626,  8.25072157]])], [array([[8.21657744, 2.89417766]]), array([[13.21657744,  7.89417766]])], [array([[7.6062021 , 2.58572457]]), array([[12.6062021 ,  7.58572457]])], [array([[8.08538599, 4.14718617]]), array([[13.08538599,  9.14718617]])], [array([[8.2964178 , 4.23985106]]), array([[13.2964178 ,  9.23985106]])], [array([[7.51487492, 4.38379575]]), array([[12.51487492,  9.38379575]])]], 'counter_examples': [[array([[78.92619947, 76.51007139]])], [array([[77.94038458, 75.34432004]])], [array([[78.1611313, 75.7708047]])], [array([[8.32559204, 8.09203017]])], [array([[8.23305693, 7.65614056]])], [array([[7.61474702, 7.0901191 ]])], [array([[9.11565942, 

In [39]:
# 3.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 3.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

146.38183903694153
{'sat_num': 14, 'counter_interval_list': [[array([[77.26896113, 68.63722976]]), array([[83.26896113, 74.63722976]])], [array([[78.22356275, 71.51300108]]), array([[84.22356275, 77.51300108]])], [array([[77.36592169, 70.04573605]]), array([[83.36592169, 76.04573605]])], [array([[77.00683442, 72.23369533]]), array([[83.00683442, 78.23369533]])], [array([[7.00423187, 0.51166286]]), array([[13.00423187,  6.51166286]])], [array([[8.01334095, 2.35233755]]), array([[14.01334095,  8.35233755]])], [array([[7.67666626, 2.75072157]]), array([[13.67666626,  8.75072157]])], [array([[7.71657744, 2.39417766]]), array([[13.71657744,  8.39417766]])], [array([[7.1062021 , 2.08572457]]), array([[13.1062021 ,  8.08572457]])], [array([[7.58538599, 3.64718617]]), array([[13.58538599,  9.64718617]])], [array([[9.66304835, 3.83638165]]), array([[15.66304835,  9.83638165]])], [array([[7.7964178 , 3.73985106]]), array([[13.7964178 ,  9.73985106]])], [array([[7.5747312 , 1.96740704]]), array([

In [17]:
# prepare the required parameters and load the functions need to be used
%run single_layer_interval_propagation.ipynb
%run simple_interval_propagation.ipynb
%run model_2_functions.ipynb

In [18]:
#***************************************************smaller pri search larger**********************************************************
# 0.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 0.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.2973687648773193
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [19]:
# 1.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.384643793106079
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [22]:
# 1.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

23.449251413345337
{'sat_num': 2, 'counter_interval_list': [[array([[78.50683442, 73.73369533]]), array([[81.50683442, 76.73369533]])], [array([[8.51487492, 5.38379575]]), array([[11.51487492,  8.38379575]])]], 'counter_examples': [[array([[79.4501938 , 76.72197658]])], [array([[9.00798099, 8.38379539]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
[[0.50046766]]
[[0.5000001]]


In [23]:
# 2.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

55.04617428779602
{'sat_num': 5, 'counter_interval_list': [[array([[79.22356275, 72.51300108]]), array([[83.22356275, 76.51300108]])], [array([[78.00683442, 73.23369533]]), array([[82.00683442, 77.23369533]])], [array([[8.58538599, 4.64718617]]), array([[12.58538599,  8.64718617]])], [array([[8.7964178 , 4.73985106]]), array([[12.7964178 ,  8.73985106]])], [array([[8.01487492, 4.88379575]]), array([[12.01487492,  8.88379575]])]], 'counter_examples': [[array([[79.227469  , 76.50518858]])], [array([[78.50683442, 76.23369533]])], [array([[9.14788599, 8.52218617]])], [array([[9.37466487, 8.73960692]])], [array([[9.52323674, 8.88367367]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
[[0.50038517]]
[[0.5471318]]
[[0.5002931]]
[[0.5000111]]
[[0.5000048]]


In [24]:
# 2.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

96.3099057674408
{'sat_num': 9, 'counter_interval_list': [[array([[78.72356275, 72.01300108]]), array([[83.72356275, 77.01300108]])], [array([[77.86592169, 70.54573605]]), array([[82.86592169, 75.54573605]])], [array([[77.50683442, 72.73369533]]), array([[82.50683442, 77.73369533]])], [array([[8.17666626, 3.25072157]]), array([[13.17666626,  8.25072157]])], [array([[8.21657744, 2.89417766]]), array([[13.21657744,  7.89417766]])], [array([[7.6062021 , 2.58572457]]), array([[12.6062021 ,  7.58572457]])], [array([[8.08538599, 4.14718617]]), array([[13.08538599,  9.14718617]])], [array([[8.2964178 , 4.23985106]]), array([[13.2964178 ,  9.23985106]])], [array([[7.51487492, 4.38379575]]), array([[12.51487492,  9.38379575]])]], 'counter_examples': [[array([[79.03606275, 76.38800108]])], [array([[78.19795294, 75.50667355]])], [array([[80.16308442, 77.42119533]])], [array([[8.82119751, 8.21165907]])], [array([[8.29470244, 7.73792766]])], [array([[7.7624521 , 7.27322457]])], [array([[9.68694849,

In [25]:
# 3.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 3.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

151.06812262535095
{'sat_num': 14, 'counter_interval_list': [[array([[77.26896113, 68.63722976]]), array([[83.26896113, 74.63722976]])], [array([[78.22356275, 71.51300108]]), array([[84.22356275, 77.51300108]])], [array([[77.36592169, 70.04573605]]), array([[83.36592169, 76.04573605]])], [array([[77.00683442, 72.23369533]]), array([[83.00683442, 78.23369533]])], [array([[7.00423187, 0.51166286]]), array([[13.00423187,  6.51166286]])], [array([[8.01334095, 2.35233755]]), array([[14.01334095,  8.35233755]])], [array([[7.67666626, 2.75072157]]), array([[13.67666626,  8.75072157]])], [array([[7.71657744, 2.39417766]]), array([[13.71657744,  8.39417766]])], [array([[7.1062021 , 2.08572457]]), array([[13.1062021 ,  8.08572457]])], [array([[7.58538599, 3.64718617]]), array([[13.58538599,  9.64718617]])], [array([[9.66304835, 3.83638165]]), array([[15.66304835,  9.83638165]])], [array([[7.7964178 , 3.73985106]]), array([[13.7964178 ,  9.73985106]])], [array([[7.5747312 , 1.96740704]]), array([

In [42]:
#***************************************************smaller pri search smaller**********************************************************
# 0.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 0.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

1.329483985900879
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [43]:
# 1.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

1.3320348262786865
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [44]:
# 1.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

21.931832313537598
{'sat_num': 2, 'counter_interval_list': [[array([[78.50683442, 73.73369533]]), array([[81.50683442, 76.73369533]])], [array([[8.51487492, 5.38379575]]), array([[11.51487492,  8.38379575]])]], 'counter_examples': [[array([[79.46665566, 76.73369533]])], [array([[9.00798122, 8.38379575]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
[[0.50000036]]
[[0.5000001]]


In [46]:
# 2.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

763.3913819789886
{'sat_num': 5, 'counter_interval_list': [[array([[79.22356275, 72.51300108]]), array([[83.22356275, 76.51300108]])], [array([[78.00683442, 73.23369533]]), array([[82.00683442, 77.23369533]])], [array([[8.58538599, 4.64718617]]), array([[12.58538599,  8.64718617]])], [array([[8.7964178 , 4.73985106]]), array([[12.7964178 ,  8.73985106]])], [array([[8.01487492, 4.88379575]]), array([[12.01487492,  8.88379575]])]], 'counter_examples': [[array([[79.23914579, 76.51299345]])], [array([[79.98208198, 77.23369533]])], [array([[9.27949733, 8.64718522]])], [array([[9.3749853 , 8.73982054]])], [array([[9.52340754, 8.88379575]])]]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}
[[0.50000036]]
[[0.50000083]]
[[0.5]]
[[0.50000054]]
[[0.5]]


In [47]:
# 2.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

101.93953037261963
{'sat_num': 9, 'counter_interval_list': [[array([[78.72356275, 72.01300108]]), array([[83.72356275, 77.01300108]])], [array([[77.86592169, 70.54573605]]), array([[82.86592169, 75.54573605]])], [array([[77.50683442, 72.73369533]]), array([[82.50683442, 77.73369533]])], [array([[8.17666626, 3.25072157]]), array([[13.17666626,  8.25072157]])], [array([[8.21657744, 2.89417766]]), array([[13.21657744,  7.89417766]])], [array([[7.6062021 , 2.58572457]]), array([[12.6062021 ,  7.58572457]])], [array([[8.08538599, 4.14718617]]), array([[13.08538599,  9.14718617]])], [array([[8.2964178 , 4.23985106]]), array([[13.2964178 ,  9.23985106]])], [array([[7.51487492, 4.38379575]]), array([[12.51487492,  9.38379575]])]], 'counter_examples': [[array([[79.75457873, 77.01300108]])], [array([[78.24204471, 75.54573605]])], [array([[80.4975083 , 77.73369533]])], [array([[8.87080136, 8.25072157]])], [array([[8.50317565, 7.89410137]])], [array([[8.18528715, 7.58572367]])], [array([[9.7949242

In [48]:
# 3.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 3.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_600, 0, 10)
results_400 = certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(orig_interval_list_400, 1, 10)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)
for i in results_600["counter_examples"]:
    print(model_2.predict(i))
for i in results_400["counter_examples"]:
    print(model_2.predict(i))

498.31285309791565
{'sat_num': 14, 'counter_interval_list': [[array([[77.26896113, 68.63722976]]), array([[83.26896113, 74.63722976]])], [array([[78.22356275, 71.51300108]]), array([[84.22356275, 77.51300108]])], [array([[77.36592169, 70.04573605]]), array([[83.36592169, 76.04573605]])], [array([[77.00683442, 72.23369533]]), array([[83.00683442, 78.23369533]])], [array([[7.00423187, 0.51166286]]), array([[13.00423187,  6.51166286]])], [array([[8.01334095, 2.35233755]]), array([[14.01334095,  8.35233755]])], [array([[7.67666626, 2.75072157]]), array([[13.67666626,  8.75072157]])], [array([[7.71657744, 2.39417766]]), array([[13.71657744,  8.39417766]])], [array([[7.1062021 , 2.08572457]]), array([[13.1062021 ,  8.08572457]])], [array([[7.58538599, 3.64718617]]), array([[13.58538599,  9.64718617]])], [array([[9.66304835, 3.83638165]]), array([[15.66304835,  9.83638165]])], [array([[7.7964178 , 3.73985106]]), array([[13.7964178 ,  9.73985106]])], [array([[7.5747312 , 1.96740704]]), array([

In [59]:
# prepare the required parameters and load the functions need to be used
%run single_layer_interval_propagation.ipynb
%run simple_interval_propagation.ipynb
%run model_2_functions.ipynb

In [60]:
# 0.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 0.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.315119981765747
{'sat_num': 1, 'counter_interval_list': [[array([[79.50683442, 74.73369533]]), array([[80.50683442, 75.73369533]])]], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [61]:
# 1.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.215898036956787
{'sat_num': 5, 'counter_interval_list': [[array([[80.22356275, 73.51300108]]), array([[82.22356275, 75.51300108]])], [array([[79.00683442, 74.23369533]]), array([[81.00683442, 76.23369533]])], [array([[9.58538599, 5.64718617]]), array([[11.58538599,  7.64718617]])], [array([[9.7964178 , 5.73985106]]), array([[11.7964178 ,  7.73985106]])], [array([[9.01487492, 5.88379575]]), array([[11.01487492,  7.88379575]])]], 'counter_examples': []}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [62]:
# 1.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.2455148696899414
{'sat_num': 21, 'counter_interval_list': [[array([[80.45068154, 71.66118366]]), array([[83.45068154, 74.66118366]])], [array([[78.76896113, 70.13722976]]), array([[81.76896113, 73.13722976]])], [array([[79.72356275, 73.01300108]]), array([[82.72356275, 76.01300108]])], [array([[83.89516096, 74.63482774]]), array([[86.89516096, 77.63482774]])], [array([[78.86592169, 71.54573605]]), array([[81.86592169, 74.54573605]])], [array([[78.50683442, 73.73369533]]), array([[81.50683442, 76.73369533]])], [array([[8.50423187, 2.01166286]]), array([[11.50423187,  5.01166286]])], [array([[9.51334095, 3.85233755]]), array([[12.51334095,  6.85233755]])], [array([[9.17666626, 4.25072157]]), array([[12.17666626,  7.25072157]])], [array([[9.21657744, 3.89417766]]), array([[12.21657744,  6.89417766]])], [array([[8.6062021 , 3.58572457]]), array([[11.6062021 ,  6.58572457]])], [array([[9.08538599, 5.14718617]]), array([[12.08538599,  8.14718617]])], [array([[10.94200639,  4.27731526]]), a

In [63]:
# 2.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.1847591400146484
{'sat_num': 39, 'counter_interval_list': [[array([[79.95068154, 71.16118366]]), array([[83.95068154, 75.16118366]])], [array([[78.26896113, 69.63722976]]), array([[82.26896113, 73.63722976]])], [array([[79.22356275, 72.51300108]]), array([[83.22356275, 76.51300108]])], [array([[83.39516096, 74.13482774]]), array([[87.39516096, 78.13482774]])], [array([[78.36592169, 71.04573605]]), array([[82.36592169, 75.04573605]])], [array([[78.00683442, 73.23369533]]), array([[82.00683442, 77.23369533]])], [array([[84.40186535, 73.00754892]]), array([[88.40186535, 77.00754892]])], [array([[8.00423187, 1.51166286]]), array([[12.00423187,  5.51166286]])], [array([[9.01334095, 3.35233755]]), array([[13.01334095,  7.35233755]])], [array([[11.14663582,  0.19527392]]), array([[15.14663582,  4.19527392]])], [array([[11.63883285,  2.10553813]]), array([[15.63883285,  6.10553813]])], [array([[8.67666626, 3.75072157]]), array([[12.67666626,  7.75072157]])], [array([[11.81936224,  2.23946763

In [64]:
# 2.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.2095022201538086
{'sat_num': 63, 'counter_interval_list': [[array([[84.77755023, 71.82784358]]), array([[89.77755023, 76.82784358]])], [array([[79.45068154, 70.66118366]]), array([[84.45068154, 75.66118366]])], [array([[77.76896113, 69.13722976]]), array([[82.76896113, 74.13722976]])], [array([[78.72356275, 72.01300108]]), array([[83.72356275, 77.01300108]])], [array([[82.89516096, 73.63482774]]), array([[87.89516096, 78.63482774]])], [array([[84.21288542, 71.70795216]]), array([[89.21288542, 76.70795216]])], [array([[77.86592169, 70.54573605]]), array([[82.86592169, 75.54573605]])], [array([[83.3605231 , 69.94816866]]), array([[88.3605231 , 74.94816866]])], [array([[77.50683442, 72.73369533]]), array([[82.50683442, 77.73369533]])], [array([[83.90186535, 72.50754892]]), array([[88.90186535, 77.50754892]])], [array([[7.50423187, 1.01166286]]), array([[12.50423187,  6.01166286]])], [array([[12.92996796, -0.03830703]]), array([[17.92996796,  4.96169297]])], [array([[14.39162782,  1.2278

In [65]:
# 3.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 3.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

1.2787723541259766
{'sat_num': 87, 'counter_interval_list': [[array([[79.88711713, 64.73883026]]), array([[85.88711713, 70.73883026]])], [array([[81.58877416, 66.08735258]]), array([[87.58877416, 72.08735258]])], [array([[84.27755023, 71.32784358]]), array([[90.27755023, 77.32784358]])], [array([[78.95068154, 70.16118366]]), array([[84.95068154, 76.16118366]])], [array([[77.26896113, 68.63722976]]), array([[83.26896113, 74.63722976]])], [array([[88.7452476 , 73.79653663]]), array([[94.7452476 , 79.79653663]])], [array([[86.95790761, 71.68927687]]), array([[92.95790761, 77.68927687]])], [array([[78.22356275, 71.51300108]]), array([[84.22356275, 77.51300108]])], [array([[82.39516096, 73.13482774]]), array([[88.39516096, 79.13482774]])], [array([[83.71288542, 71.20795216]]), array([[89.71288542, 77.20795216]])], [array([[77.36592169, 70.04573605]]), array([[83.36592169, 76.04573605]])], [array([[82.8605231 , 69.44816866]]), array([[88.8605231 , 75.44816866]])], [array([[77.00683442, 72.23

In [69]:
# prepare the required parameters and load the functions need to be used
%run single_layer_interval_propagation.ipynb
%run simple_interval_propagation.ipynb
%run model_2_functions.ipynb

In [70]:
# 0.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 0.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

45.75382614135742
{'sat_num': 1, 'counter_interval_list': [[array([[79.50683442, 74.73369533]]), array([[80.50683442, 75.73369533]])]], 'counter_examples': [None]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [72]:
# 1.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

50.12611198425293
{'sat_num': 5, 'counter_interval_list': [[array([[80.22356275, 73.51300108]]), array([[82.22356275, 75.51300108]])], [array([[79.00683442, 74.23369533]]), array([[81.00683442, 76.23369533]])], [array([[9.58538599, 5.64718617]]), array([[11.58538599,  7.64718617]])], [array([[9.7964178 , 5.73985106]]), array([[11.7964178 ,  7.73985106]])], [array([[9.01487492, 5.88379575]]), array([[11.01487492,  7.88379575]])]], 'counter_examples': [None, None, None, None, None]}
{'sat_num': 0, 'counter_interval_list': [], 'counter_examples': []}


In [73]:
# 1.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 1.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

53.474045515060425
{'sat_num': 21, 'counter_interval_list': [[array([[80.45068154, 71.66118366]]), array([[83.45068154, 74.66118366]])], [array([[78.76896113, 70.13722976]]), array([[81.76896113, 73.13722976]])], [array([[79.72356275, 73.01300108]]), array([[82.72356275, 76.01300108]])], [array([[83.89516096, 74.63482774]]), array([[86.89516096, 77.63482774]])], [array([[78.86592169, 71.54573605]]), array([[81.86592169, 74.54573605]])], [array([[78.50683442, 73.73369533]]), array([[81.50683442, 76.73369533]])], [array([[8.50423187, 2.01166286]]), array([[11.50423187,  5.01166286]])], [array([[9.51334095, 3.85233755]]), array([[12.51334095,  6.85233755]])], [array([[9.17666626, 4.25072157]]), array([[12.17666626,  7.25072157]])], [array([[9.21657744, 3.89417766]]), array([[12.21657744,  6.89417766]])], [array([[8.6062021 , 3.58572457]]), array([[11.6062021 ,  6.58572457]])], [array([[9.08538599, 5.14718617]]), array([[12.08538599,  8.14718617]])], [array([[10.94200639,  4.27731526]]), a

In [74]:
# 2.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

52.02173352241516
{'sat_num': 39, 'counter_interval_list': [[array([[79.95068154, 71.16118366]]), array([[83.95068154, 75.16118366]])], [array([[78.26896113, 69.63722976]]), array([[82.26896113, 73.63722976]])], [array([[79.22356275, 72.51300108]]), array([[83.22356275, 76.51300108]])], [array([[83.39516096, 74.13482774]]), array([[87.39516096, 78.13482774]])], [array([[78.36592169, 71.04573605]]), array([[82.36592169, 75.04573605]])], [array([[78.00683442, 73.23369533]]), array([[82.00683442, 77.23369533]])], [array([[84.40186535, 73.00754892]]), array([[88.40186535, 77.00754892]])], [array([[8.00423187, 1.51166286]]), array([[12.00423187,  5.51166286]])], [array([[9.01334095, 3.35233755]]), array([[13.01334095,  7.35233755]])], [array([[11.14663582,  0.19527392]]), array([[15.14663582,  4.19527392]])], [array([[11.63883285,  2.10553813]]), array([[15.63883285,  6.10553813]])], [array([[8.67666626, 3.75072157]]), array([[12.67666626,  7.75072157]])], [array([[11.81936224,  2.23946763]

In [75]:
# 2.5 perturbation
# get the 1000 intervals by using l infinity
perturbation = 2.5
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

47.61380910873413
{'sat_num': 63, 'counter_interval_list': [[array([[84.77755023, 71.82784358]]), array([[89.77755023, 76.82784358]])], [array([[79.45068154, 70.66118366]]), array([[84.45068154, 75.66118366]])], [array([[77.76896113, 69.13722976]]), array([[82.76896113, 74.13722976]])], [array([[78.72356275, 72.01300108]]), array([[83.72356275, 77.01300108]])], [array([[82.89516096, 73.63482774]]), array([[87.89516096, 78.63482774]])], [array([[84.21288542, 71.70795216]]), array([[89.21288542, 76.70795216]])], [array([[77.86592169, 70.54573605]]), array([[82.86592169, 75.54573605]])], [array([[83.3605231 , 69.94816866]]), array([[88.3605231 , 74.94816866]])], [array([[77.50683442, 72.73369533]]), array([[82.50683442, 77.73369533]])], [array([[83.90186535, 72.50754892]]), array([[88.90186535, 77.50754892]])], [array([[7.50423187, 1.01166286]]), array([[12.50423187,  6.01166286]])], [array([[12.92996796, -0.03830703]]), array([[17.92996796,  4.96169297]])], [array([[14.39162782,  1.22780

In [76]:
# 3.0 perturbation
# get the 1000 intervals by using l infinity
perturbation = 3.0
orig_interval_list_600 = get_l_infinity_list(orig_point_list_600, perturbation)
orig_interval_list_400 = get_l_infinity_list(orig_points_list_400, perturbation)

s_time = time.time()
results_600 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_600, 0)
results_400 = certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(orig_interval_list_400, 1)
used_time=time.time() - s_time
print(used_time)
print(results_600)
print(results_400)

47.27856230735779
{'sat_num': 87, 'counter_interval_list': [[array([[79.88711713, 64.73883026]]), array([[85.88711713, 70.73883026]])], [array([[81.58877416, 66.08735258]]), array([[87.58877416, 72.08735258]])], [array([[84.27755023, 71.32784358]]), array([[90.27755023, 77.32784358]])], [array([[78.95068154, 70.16118366]]), array([[84.95068154, 76.16118366]])], [array([[77.26896113, 68.63722976]]), array([[83.26896113, 74.63722976]])], [array([[88.7452476 , 73.79653663]]), array([[94.7452476 , 79.79653663]])], [array([[86.95790761, 71.68927687]]), array([[92.95790761, 77.68927687]])], [array([[78.22356275, 71.51300108]]), array([[84.22356275, 77.51300108]])], [array([[82.39516096, 73.13482774]]), array([[88.39516096, 79.13482774]])], [array([[83.71288542, 71.20795216]]), array([[89.71288542, 77.20795216]])], [array([[77.36592169, 70.04573605]]), array([[83.36592169, 76.04573605]])], [array([[82.8605231 , 69.44816866]]), array([[88.8605231 , 75.44816866]])], [array([[77.00683442, 72.233